<a href="https://colab.research.google.com/github/Tecindes/PrecioCasas/blob/main/precios_de_inmuebles_regresion_lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Precificación inmobiliaria**

**Objetivo**: Estimar los precios de los inmuebles.
 * Identificar los aspectos que tienen más influencia en la precificación de los inmuebles.
 * Entender cuál de todos estos aspectos es el más relevante.
 * Precificar un inmueble nuevo.

Base de datos simplificada e inspirada en [House Prices](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques)



![](https://i.imgur.com/A8X79yq.jpeg)


# 1. Ajustando una recta

## Conociendo los datos

In [1]:
#Leyendo los datos
import pandas as pd
datos=pd.read_csv('/content/precios_casas.csv')
datos.head(2)

FileNotFoundError: [Errno 2] No such file or directory: '/content/precios_casas.csv'

In [ ]:
#Cuáles factores fueron colectados
datos.info()

In [ ]:
datos=datos.drop(columns='Id')
datos.columns

## Correlación

**¿Cuáles factores están relacionados con el precio de la casa? ¿Cómo es esa relación?**

El coeficiente de correlación de Pearson nos permite medir la relación lineal entre variables, ofreciendo una escala que varía de -1 a 1, la cual interpretamos según su intensidad y dirección:

* **-1**: correlación negativa perfecta: a medida que una variable aumenta, la otra disminuye.  
* **0**: no hay relación lineal entre las variables.  
* **1**: correlación positiva perfecta: a medida que una variable aumenta, la otra también aumenta.  



In [ ]:
# Correlación
corr=datos.corr()

In [ ]:
corr['precio_de_venta']

In [ ]:
# Actividad
# Cuáles factores están más correlacionados?
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Generar una máscara para el triángulo superior
mascara = np.zeros_like(corr, dtype=bool)
mascara[np.triu_indices_from(mascara)] = True

# Configurar la figura de matplotlib
f, ax = plt.subplots(figsize=(11, 9))

# Generar el mapa de calor (heatmap)
cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr, mask=mascara, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .5})

# Mostrar el mapa de calor (heatmap)
plt.show()

## Relacionando variables

In [ ]:
# importando los paquetes para la visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [ ]:
# Cómo se relacionan el área construída y el precio del inmueble?
plt.scatter(datos['area_primer_piso'], datos['precio_de_venta'])
plt.title('Relación entre precio y área')
plt.xlabel('Área en m2')
plt.ylabel('Precio en USD')
plt.show()

In [ ]:
# Aparentemente mientras mayor es el área del primer piso, mayor es el precio de la casa.
# E si quisieramos trazar una línea que nos permita describir este comportamiento?
plt.scatter(datos['area_primer_piso'], datos['precio_de_venta'])
plt.axline(xy1=(40,300000), xy2=(175,1500000),color='red')
plt.title('Relación entre precio y área')
plt.xlabel('Área en m2')
plt.ylabel('Precio en USD')
plt.show()

## Mejor recta

In [ ]:
# Cuál es la recta que mejor se ajusta a esta relación
px.scatter(datos,x='area_primer_piso',y='precio_de_venta',trendline_color_override='red',trendline='ols')


# 2. Explicando la recta
Ajustamos una recta entre los $m^2$ del primer piso y el precio de la casa. Queremos explicar el precio de la casa a partir de su tamaño, por eso decimos que:

* Variable explicativa/independiente: Área del primer piso  
* Variable de respuesta/dependiente: Precio de la casa  

In [ ]:
#Quién es nuestra variable de respuesta?
sns.displot(datos,x='precio_de_venta',kde=True,color='green')
plt.title('Distribución del precio de venta de las casas')
plt.xlabel('Precio en USD')
plt.show()


### Separando datos para entrenamiento y para prueba

El conjunto de **entrenamiento** se utiliza para ajustar el modelo, mientras que el conjunto de **prueba** se usa para evaluar su desempeño al predecir precios de viviendas no vistos durante el entrenamiento, lo que ayuda a la generalización del modelo.

In [ ]:
# import train_test_split
from sklearn.model_selection import train_test_split
y=datos['precio_de_venta']
x=datos.drop(columns='precio_de_venta')
X_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=230) # garantiza reproductivilidad de la segregación de los datos

In [ ]:
#Datos de entrenamiento para usar la fórmula
df_train=pd.DataFrame(X_train)
df_train['precio_de_venta']=y_train


In [ ]:
# import ols
from statsmodels.formula.api import ols

In [ ]:
# ajustando el primer modelo
modelo_0=ols('precio_de_venta ~  area_primer_piso',data=df_train).fit()

## Analizando los coeficientes

(intensidad, dirección y significancia)




In [ ]:
# visualizando los parámetros
modelo_0.params

In [ ]:
# El resumen del modelo
print(modelo_0.summary())


- El **punto de intercepción** es el valor esperado del precio de la casa cuando todas las demás variables son cero. En este caso, cuando todas las demás variables, incluida el área del primer piso, son cero, el precio esperado de la casa es de $145.196,40. No siempre se tiene una interpretación práctica para este número.

- El **efecto individual** del área es de \$6.833,97. Esto indica que por cada metro cuadrado adicional al área del primer piso, se espera que el precio de la casa aumente en promedio \$6.833,97.

<img src="https://i.ibb.co/CJwfMgW/coeficientes.png" alt="coeficientes" border="0">


## Explicabilidad del modelo

**¿En qué medida la variación del área explica los diversos precios de las casas?**  

En este caso, recurrimos a la métrica R², el coeficiente de determinación. El R² varía de 0 a 1, donde 1 indica un ajuste perfecto del modelo a los datos, es decir, todas las variaciones en la variable dependiente son explicadas por las variables independientes en el modelo. Por otro lado, un R² de 0 indica que el modelo no explica ninguna variabilidad en la variable dependiente.

In [ ]:
# observando el R²
modelo_0.rsquared

## Entendiendo el resíduo

In [ ]:
# Quiénes son los residuos?
modelo_0.resid

In [ ]:
# Cómo están distribuidos
modelo_0.resid.hist()
plt.title('Distribución de los residuos')
plt.xlabel('Residuos')
plt.show()


<img src="https://i.ibb.co/zGTB35V/residuos.png" alt="residuos" border="0">

## Obteniendo el R² de la previsión

In [ ]:
# definiendo la 'Y' prevista
y_predict=modelo_0.predict(x_test)

In [ ]:
# importando el r2_score
from sklearn.metrics import r2_score
print(f'El coeficiente de determinación R² para el modelo con los datos de prueba es de: {round(r2_score(y_test,y_predict),2)}')


In [ ]:
# mostrando el resultado de r²


# 3. Añadiendo otras características

El modelo con un solo factor nos mostró un R² de 0.38, es decir, aproximadamente el 38% de la variación observada en los precios de las casas puede explicarse por la variación en el área.  
Esto indica que aún existe una cantidad significativa de variación que no está siendo capturada por este modelo específico. Analizaremos otros factores para explicar el precio de las casas.

## Analizando los factores

In [ ]:
# cuáles otras características podrían explicar el precio de los inmuebles?
sns.pairplot(datos)

In [ ]:
#Vamos a analizar solamente y_vars='precio_de_venta'
sns.pairplot(datos,y_vars=['area_primer_piso','area_segundo_piso','cantidad_banos'],x_vars=['precio_de_venta'])

## Añadiendo factores al modelo

In [ ]:
# importando la api de statsmodels
import statsmodels.api as sm


In [ ]:
# Añadiendo una constante
X_train=sm.add_constant(X_train)
X_train.head()

In [ ]:
# Creando el modelo de regresión (con todas las variables): saturado
X_train.columns
modelo_1=sm.OLS(y_train,X_train[['const', 'area_primer_piso', 'tiene_segundo_piso', 'area_segundo_piso',
       'cantidad_banos', 'capacidad_carros_garage',
       'calidad_de_cocina_excelente']]).fit()

In [ ]:
# Modelo sin el área del segundo piso
modelo_2=sm.OLS(y_train,X_train[['const', 'area_primer_piso', 'tiene_segundo_piso',
                                 'cantidad_banos', 'capacidad_carros_garage',
                                 'calidad_de_cocina_excelente']]).fit()

In [ ]:
# Modelo sin la información del garage
# Modelo sin el área del segundo piso
modelo_3=sm.OLS(y_train,X_train[['const', 'area_primer_piso', 'tiene_segundo_piso',
                                 'cantidad_banos',
                                 'calidad_de_cocina_excelente']]).fit()

In [ ]:
# Resumen del modelos
modelos=[modelo_0,modelo_1,modelo_2,modelo_3]
for i,j in enumerate(modelos):
  print(f'************************************************\n**** El modelo {i} tiene el siguiente resumen ****\n************************************************')
  print(j.summary(),'\n\n')


In [ ]:
for i,j in enumerate(modelos):
  print(f'El coeficiente de determinación R² para el modelo {i} es de: {round(j.rsquared,2)}')

## Comparando los modelos
Cuál es el modelo?


In [ ]:
# Cuántos parámetros tiene el modelo?
modelo_3.params

# 4. Precificando las casas

## Obteniendo el R² de la previsión

In [ ]:
# Añadiendo una constante en X_test
x_test=sm.add_constant(x_test)
x_test.head(2)

In [ ]:
# Previsión con el modelo 3
prevision_3=modelo_3.predict(x_test[['const','area_primer_piso','tiene_segundo_piso','cantidad_banos','calidad_de_cocina_excelente']])

In [ ]:
# Cuál es el r² de la previsión?
print(f'El coeficiente R2 de la base de previsión es:{modelo_3.rsquared.round(2)}')

In [ ]:
# Cuál es el R² del entrenamiento?
print(f'El coeficiente R2 con respecto  a la base de entrenamiento es:{round(r2_score(y_test,prevision_3),2)}')

## Precificando una casa

<img src="https://i.ibb.co/SvG4V4j/costo.png" alt="costo" border="0">

In [ ]:
#Nuevo inmueble
nuevo_inmueble=pd.DataFrame({'const':[1],
                             'area_primer_piso':[120],
                             'tiene_segundo_piso':[1],
                             'cantidad_banos':[2],
                             'caldiad_de_cocina_excelente':[0]})

In [ ]:
# Cuál es el precio del inmueble de acuerdo con el modelo 0?
modelo_0.predict(nuevo_inmueble['area_primer_piso']).round(2)

In [ ]:
#Cuál es el precio del inmueble de acuerdo con el modelo 3?
modelo_3.predict(nuevo_inmueble).round(2)

In [ ]:
modelo_3.params

## Precificando varias casas

<img src="https://i.imgur.com/i0skFlB.jpeg" width="600"/>

In [ ]:
# Leyendo varias casas?
nuevas_casas=pd.read_csv('/content/nuevas_casas.csv',sep=';')
nuevas_casas.head(2)

In [ ]:
nuevas_casas.drop(columns='Casa',inplace=True)
nuevas_casas.head(2)

In [ ]:
# Añadiendo una constante
nuevas_casas=sm.add_constant(nuevas_casas)
nuevas_casas.head(2)

In [ ]:
# Cuál es el precio de estas nuevas casas?
precio_casas = pd.DataFrame()
precio_casas['Preciode venta'] = modelo_3.predict(nuevas_casas).round(2)
precio_casas
nuevas_casas['Preciode venta'] = modelo_3.predict(nuevas_casas).round(2)
nuevas_casas

In [ ]:
nuevo_inmueble = pd.DataFrame({'const':[1],
                               'area_primer_piso':[98],
                               'existe_segundo_piso': [0],
                               'cantidad_baños': [1],
                               'calidad_de_la_cocina_Excelente': [1]
                               })
modelo_0.predict(nuevo_inmueble['area_primer_piso'])
modelo_3.predict(nuevo_inmueble)[0]

In [ ]:
import pickle

# Nombre del archivo donde se guardará el modelo
nombre_archivo = 'modelo_regresion_lineal.pkl'

# Guardar el modelo en un archivo usando pickle
with open(nombre_archivo, 'wb') as archivo:
    pickle.dump(modelo_0, archivo)

In [ ]:
# Cargar el modelo de vuelta del archivo
with open(nombre_archivo, 'rb') as archivo:
    modelo_cargado = pickle.load(archivo)

# 5. Investigando el modelo





## Multicolinearidad

Cuando dos o más cosas que estás intentando estudiar son tan parecidas que es difícil determinar cuánto está influyendo cada una de ellas en el resultado.

In [ ]:
# importar el VIF de statsmodel


In [ ]:
# VIF 1


In [ ]:
# VIF 3


## Análisis de los residuos

Residuos: la discrepancia entre los valores reales y los valores que el modelo predice para los mismos puntos de datos.

<img src="https://i.ibb.co/9bmySbm/heterocedasticidad.png" alt="heterocedasticidad" border="0">